In [1]:
import re
import os
import glob
import zipfile
import requests
from urllib.request import urlretrieve
import json
import pandas as pd

In [2]:
article_id = 14096681 
url = f"https://api.figshare.com/v2/articles/{article_id}"
headers = {"Content-Type": "application/json"}
output_directory = "figshareaustralia/"

response = requests.request("GET", url, headers=headers)
data = json.loads(response.text) 
files = data["files"]             
files

[{'id': 26579150,
  'name': 'daily_rainfall_2014.png',
  'size': 58863,
  'is_link_only': False,
  'download_url': 'https://ndownloader.figshare.com/files/26579150',
  'supplied_md5': 'fd32a2ffde300a31f8d63b1825d47e5e',
  'computed_md5': 'fd32a2ffde300a31f8d63b1825d47e5e'},
 {'id': 26579171,
  'name': 'environment.yml',
  'size': 192,
  'is_link_only': False,
  'download_url': 'https://ndownloader.figshare.com/files/26579171',
  'supplied_md5': '060b2020017eed93a1ee7dd8c65b2f34',
  'computed_md5': '060b2020017eed93a1ee7dd8c65b2f34'},
 {'id': 26586554,
  'name': 'README.md',
  'size': 5422,
  'is_link_only': False,
  'download_url': 'https://ndownloader.figshare.com/files/26586554',
  'supplied_md5': '61858c6cc0e6a6d6663a7e4c75bbd88c',
  'computed_md5': '61858c6cc0e6a6d6663a7e4c75bbd88c'},
 {'id': 26766812,
  'name': 'data.zip',
  'size': 814041183,
  'is_link_only': False,
  'download_url': 'https://ndownloader.figshare.com/files/26766812',
  'supplied_md5': 'b517383f76e77bd03755a63a8f

In [5]:
%%time
files_to_dl = ["data.zip"] 
for file in files:
    if file["name"] in files_to_dl:
        os.makedirs(output_directory, exist_ok=True)
        urlretrieve(file["download_url"], output_directory + file["name"])

CPU times: total: 12.9 s
Wall time: 58min 36s


In [6]:
%%time
with zipfile.ZipFile(os.path.join(output_directory, "data.zip"), 'r') as f:
    f.extractall(output_directory)

CPU times: total: 44.8 s
Wall time: 45 s


In [12]:
%%sh
ls -ltr figshareaustralia/

total 6016504
-rw-r--r-- 1 plevc 197609 814041183 Mar 28 14:10 data.zip
-rw-r--r-- 1 plevc 197609  95376895 Mar 28 14:10 MPI-ESM-1-2-HAM_daily_rainfall_NSW.csv
-rw-r--r-- 1 plevc 197609  94960113 Mar 28 14:10 AWI-ESM-1-1-LR_daily_rainfall_NSW.csv
-rw-r--r-- 1 plevc 197609  82474546 Mar 28 14:10 NorESM2-LM_daily_rainfall_NSW.csv
-rw-r--r-- 1 plevc 197609 127613760 Mar 28 14:10 ACCESS-CM2_daily_rainfall_NSW.csv
-rw-r--r-- 1 plevc 197609 232118894 Mar 28 14:10 FGOALS-f3-L_daily_rainfall_NSW.csv
-rw-r--r-- 1 plevc 197609 330360682 Mar 28 14:10 CMCC-CM2-HR4_daily_rainfall_NSW.csv
-rw-r--r-- 1 plevc 197609 254009247 Mar 28 14:10 MRI-ESM2-0_daily_rainfall_NSW.csv
-rw-r--r-- 1 plevc 197609 235661418 Mar 28 14:10 GFDL-CM4_daily_rainfall_NSW.csv
-rw-r--r-- 1 plevc 197609 294260911 Mar 28 14:10 BCC-CSM2-MR_daily_rainfall_NSW.csv
-rw-r--r-- 1 plevc 197609 295768615 Mar 28 14:10 EC-Earth3-Veg-LR_daily_rainfall_NSW.csv
-rw-r--r-- 1 plevc 197609 328852379 Mar 28 14:10 CMCC-ESM2_daily_rainfall_NSW.csv

In [3]:
%%time
files = glob.glob('figshareaustralia/*.csv')
df = pd.concat((pd.read_csv(file, index_col=0)
                .assign(model=file.split('\\')[1].split('_daily')[0])
                for file in files)
              )
df.to_csv("figshareaustralia/combined_data.csv")

CPU times: total: 20min 44s
Wall time: 20min 51s


In [4]:
df.head(3)

,lat_min,lat_max,lon_min,lon_max,rain (mm/day),model
time,,,,,,
1889-01-01 12:00:00,-36.25,-35.0,140.625,142.5,3.293256e-13,ACCESS-CM2
1889-01-02 12:00:00,-36.25,-35.0,140.625,142.5,0.000000e+00,ACCESS-CM2
1889-01-03 12:00:00,-36.25,-35.0,140.625,142.5,0.000000e+00,ACCESS-CM2
